In [1]:
import pickle
from datetime import datetime
from os import mkdir
from os.path import join, exists

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from lightgbm import LGBMClassifier

In [2]:
train_features = pd.read_pickle('../dataset/all_feature.pkl').reset_index(drop=True)
train_features.shape

(1080976, 265)

In [3]:
model_path = '../dataset/lightgbmmodel.pkl'

In [4]:
luck = 123456

In [5]:
model = LGBMClassifier(nthread=4, silent=False, objective= 'binary', seed=luck)

In [6]:
parameters = {
#     'gamma': [0.05],
    #     'n_estimators': [800],
    'max_depth': [2, 3, 4],
#     'learning_rate': [0.15, 0.3],
        'subsample': [0.9],
        'colsample_bytree': [0.9],
    #     'reg_alpha': [0, 1, 5],
    #     'reg_lambda': [0, 1, 4],
}

In [7]:
use_columns = [
    x for x in train_features.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid',
        'orderlabel'
    ]
]

In [8]:
cv = model_selection.ShuffleSplit(n_splits=1, test_size=0.2, random_state=luck)
clf = model_selection.GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=True)

In [9]:
clf_feture = train_features.sample(1000, random_state=luck)

In [10]:
%time clf.fit(clf_feture[use_columns], clf_feture['orderlabel'])

Fitting 1 folds for each of 3 candidates, totalling 3 fits
CPU times: user 161 ms, sys: 48.1 ms, total: 209 ms
Wall time: 430 ms


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.3s finished


GridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=123456, test_size=0.2, train_size=None),
       error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=4,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=123456, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [2, 3, 4], 'colsample_bytree': [0.9], 'subsample': [0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [25]:
# # clf.best_params_['n_estimators'] = 
clf.best_params_['max_depth'] = 12
# clf.best_params_['reg_lambda'] = 0.5
clf.best_params_['subsample'] = 0.7
clf.best_params_['colsample_bytree'] = 0.7
# clf.best_params_['n_estimators'] = 1000
clf.best_params_['learning_rate'] = 0.1
clf.best_params_['num_leaves'] = 24
# clf.best_params_['reg_alpha'] = 1
# clf.best_params_['min_data_in_leaf'] = 3

# clf.best_params_['min_child_weight'] = 1

In [26]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=4000,
    **clf.best_params_)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, learning_rate=0.1,
        max_bin=255, max_depth=2, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=4000, nthread=4, num_leaves=6,
        objective='binary', reg_alpha=0, reg_lambda=0, seed=123456,
        silent=False, subsample=0.7, subsample_for_bin=50000,
        subsample_freq=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    train_features[use_columns],
    train_features['orderlabel'],
    test_size=0.33,
    random_state=42)

In [27]:
model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    early_stopping_rounds=10,
    eval_metric='auc',
    verbose=True, )

[1]	training's auc: 0.715686	valid_1's auc: 0.71463
Train until valid scores didn't improve in 5 rounds.
[2]	training's auc: 0.798386	valid_1's auc: 0.794022
[3]	training's auc: 0.827174	valid_1's auc: 0.823519
[4]	training's auc: 0.827585	valid_1's auc: 0.824023
[5]	training's auc: 0.844787	valid_1's auc: 0.842096
[6]	training's auc: 0.846628	valid_1's auc: 0.843199
[7]	training's auc: 0.853824	valid_1's auc: 0.850673
[8]	training's auc: 0.853038	valid_1's auc: 0.849359
[9]	training's auc: 0.856178	valid_1's auc: 0.852489
[10]	training's auc: 0.855538	valid_1's auc: 0.851531
[11]	training's auc: 0.855492	valid_1's auc: 0.851569
[12]	training's auc: 0.856232	valid_1's auc: 0.853156
[13]	training's auc: 0.8562	valid_1's auc: 0.85272
[14]	training's auc: 0.856864	valid_1's auc: 0.853731
[15]	training's auc: 0.857927	valid_1's auc: 0.853983
[16]	training's auc: 0.857909	valid_1's auc: 0.85474
[17]	training's auc: 0.859616	valid_1's auc: 0.856141
[18]	training's auc: 0.860587	valid_1's auc

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, learning_rate=0.1,
        max_bin=255, max_depth=2, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=4000, nthread=4, num_leaves=6,
        objective='binary', reg_alpha=0, reg_lambda=0, seed=123456,
        silent=False, subsample=0.7, subsample_for_bin=50000,
        subsample_freq=1)

In [28]:
def get_score(model, X_train, train_features):
    result = model.predict_proba(X_train)[:, 1]
    train_features['prob'] = np.nan
    train_features.loc[X_train.index, 'prob'] = result
    train_x = train_features.loc[X_train.index, ['orderid', 'prob', 'orderlabel']]
    train_x.sort_values('prob', ascending=False, inplace=True)
    finall_result_x = train_x.drop_duplicates(['orderid'])
    train_score = finall_result_x.orderlabel.mean()
    print(train_score)
    return train_score

In [29]:
train_score = get_score(model, X_train, train_features)

0.32124472784


In [30]:
test_score = get_score(model, X_test, train_features)

0.19883493838


In [31]:
if not exists('models'):
    mkdir('models')
model_importance_path = join('models', datetime.now().strftime('%d-%H%M-importance.txt'))
model_importance_path

'models/31-2339-importance.txt'

In [32]:
importance_df = pd.DataFrame(model.feature_importances_, index=use_columns)

importance_df.sort_values(0, ascending=False, inplace=True)
importance_df.to_csv(model_importance_path)

In [33]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in clf.best_params_.items()])
parms

'max_depth: 2,colsample_bytree: 0.7,subsample: 0.7,learning_rate: 0.1,num_leaves: 6'

In [34]:
print(datetime.now().strftime('%d-%H-%M:'),
      'online', '??',
     'test_score', test_score,
     'train_score', train_score,
     'model_train', model.best_score['training']['auc'],
     'model_test', model.best_score['valid_1']['auc'],
     'n', model.best_iteration,
     'params', parms, end='\n', sep='  ')

31-23-39:  online  ??  test_score  0.19883493838  train_score  0.32124472784  model_train  0.917111235022  model_test  0.912981448034  n  396  params  max_depth: 2,colsample_bytree: 0.7,subsample: 0.7,learning_rate: 0.1,num_leaves: 6


In [35]:
print(datetime.now().strftime('%d-%H-%M:'),
      'online', '??',
     'test_score', test_score,
     'train_score', train_score,
     'model_train', model.best_score['training']['auc'],
     'model_test', model.best_score['valid_1']['auc'],
     'n', model.best_iteration,
     'params', parms, end='\n', sep=',', file=open('lgb_result.txt', 'a+'))

In [36]:
pickle.dump(model, open(model_path, 'wb'))